In [ ]:
import numpy as np
import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

In [ ]:
# Connect to the EFD 

# efd_client = EfdClient('summit_efd') 
efd_client = EfdClient('usdf_efd')

In [ ]:
# Set a range of dayobs values to search - 
day_obs_min = "2024-11-04"
t_start = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
day_obs_max = "2024-11-04"
t_end = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc') + TimeDelta(0.8, format='jd')
print(f"Querying the EFD from {t_start.iso} to {t_end.iso}")

In [ ]:
# Query any EFD topic for the timespan day_obs_min to day_obs_max, when you don't already know the fields
# topic = lsst.sal.ScriptQueue.command_add
# fields = await efd_client.get_fields(topic)
# fields = [f for f in fields if 'private' not in f and f != 'name' and f!= "duration"]
# dd = await efd_client.select_time_series(topic, fields, tstart, tend)
# or top 5 .. 
# dd = await efd_client.select_top_n(topic, fields, 5)

In [ ]:
# from https://github.com/lsst-ts/ts_xml/blob/develop/python/lsst/ts/xml/enums -- import would be better 
# Informational reference point .. 
import enum

class ScriptProcessState(enum.IntEnum):
    """ScriptQueue script.processState event constants."""

    UNKNOWN = 0
    LOADING = 1
    CONFIGURED = 2
    RUNNING = 3
    DONE = 4
    LOADFAILED = 5
    CONFIGURE_FAILED = 6
    TERMINATED = 7
    CONFIGUREFAILED = 6  # deprecated alias for CONFIGURE_FAILED

class ScriptState(enum.IntEnum):
    """ScriptState constants."""

    UNKNOWN = 0
    UNCONFIGURED = 1
    CONFIGURED = 2
    RUNNING = 3
    PAUSED = 4
    ENDING = 5
    STOPPING = 6
    FAILING = 7
    DONE = 8
    STOPPED = 9
    FAILED = 10
    CONFIGURE_FAILED = 11

In [ ]:
# find when this block was executed - should be in salindex 1 (although some daytime calibration happens in 3)

# # This will find JSON BLOCKS added to the queue, but not scripts executed that did not use a JSON BLOCK
topic = 'lsst.sal.Scheduler.command_addBlock'
fields = ['executionId', 'id', 'salIndex']
json_blocks = await efd_client.select_time_series(topic, fields, t_start, t_end)

# And it turns out this is true for the ScriptQueue too
topic = 'lsst.sal.ScriptQueue.command_add'
fields = ['config', 'descr', 'salIndex', 'path', 'block', 'startBlock']
scriptqueue_added = await efd_client.select_time_series(topic, fields, t_start, t_end)

# It's larger, much more data, but search the script configure instead because that will find both JSON BLOCKs and scripts
# This gets us more information about the script parameters 
topic = 'lsst.sal.Script.command_configure'
fields = ['blockId', 'config',' executionId', 'salIndex']
scriptconfig = await efd_client.select_time_series(topic, fields, t_start, t_end)
scriptconfig["efd_config_time"] = scriptconfig.index.copy()

In [ ]:
# To see what actual was configured or ran, we could look at the script queue 'queue' itself .. but it's pretty opaque 
# this is probably mostly useful for seeing - at a given time, what was running now and what was running just before
# but you'll still have to trace the salindex values into the scriptSalIndex in the scriptqueue.logevent_script 
topic = 'lsst.sal.ScriptQueue.logevent_queue'
fields = ['ScriptQueueID', 'currentSalIndex', 'enabled', 'priority', 'running', 'length', 'pastLength', 'pastSalIndices0', 'salIndices0',  'salIndex']
scriptqueue = await efd_client.select_time_series(topic, fields, t_start, t_end)

# So the best way to see all of the scripts that have been configured or run is here
topic = 'lsst.sal.ScriptQueue.logevent_script'
fields = await efd_client.get_fields(topic)
fields = ['blockId', 'path', 'processState', 'scriptState', 'salIndex', 'scriptSalIndex', 
         'timestampProcessStart', 'timestampConfigureStart', 'timestampConfigureEnd', 'timestampRunStart', 'timestampProcessEnd' ]
scripts = await efd_client.select_time_series(topic, fields, t_start, t_end)

In [ ]:
display(scriptqueue.head())

In [ ]:
script_status = scripts.query('scriptState > 2').groupby('scriptSalIndex').agg({'path': 'first', 'salIndex': 'max', 'scriptState': 'unique', 'processState': 'unique', 
                                                                      'timestampProcessStart': 'max', 'timestampConfigureStart': 'max', 'timestampConfigureEnd': 'max', 
                                                                      'timestampRunStart': 'max', 'timestampProcessEnd': 'max'}).sort_values(by='timestampRunStart')
script_status['timestamp_run'] = Time(run['timestampRunStart'], format='unix').iso
cols = ['path', 'config', 'salIndex', 'salIndex_r', 'scriptState', 'processState', 'timestampProcessStart', 'timestampRunStart', 'timestampProcessEnd', 'timestamp_run', 'efd_config_time']
script_info = pd.merge(script_status, scriptconfig, left_on='scriptSalIndex', right_on='salIndex', suffixes=['_r', ''])[cols]

In [ ]:
# But this is still ALL of the scripts that ran in the night, which include scripts to slew or recover from fault
# This might work to pull out only "do this kind of program" ... 
ignore_scripts = ['set_summary_state.py', 'run_command.py', 'maintel/laser_tracker', 
                  'maintel/mtrotator', 'maintel/mtdome', 'maintel/home', 'maintel/m1m3/', 'auxtel']
#script_info.query('path.str.contains("run_command.py")')['config'].unique()
sub = script_info
for ig in ignore_scripts:
    sub = sub.query('~path.str.contains(@ig)')
sub.path.unique()

In [ ]:
with option_context('display.max_colwidth', None):
    display(HTML(sub[cols].to_html()))